In [1]:
#Mount Drive if running in Colab
from sys import path
import os

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'

  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)
else:
  root_PATH = os.path.abspath("../..")

%load_ext autoreload
%autoreload 2

module_path = os.path.abspath(os.path.join('../../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_cleaner import DataCleaner
from data_processor import DataProcessor

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [3]:
#Configuration variables
#remove under-represented categories below this threshold
underrepresented_threshold = 3000

In [4]:
#Load datasets
train_df = pd.read_csv(root_PATH + '/data/train.csv')
validation_df = pd.read_csv(root_PATH + '/data/validation.csv')

#Convert topics column to list
train_df["Topic"] = train_df["Topic"].apply(eval)
validation_df["Topic"] = validation_df["Topic"].apply(eval)

In [5]:
#Clean datasets for classical methods
train_df['Conversation'] = train_df['Conversation'].map(lambda row : DataCleaner.clean_text_for_classical_methods(str(row)))
validation_df['Conversation'] = validation_df['Conversation'].map(lambda row : DataCleaner.clean_text_for_classical_methods(str(row)))

In [6]:
#Generate boolean masks for our datasets
train_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(train_df)
validation_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(validation_df)

#Remove underrepresented topics
underrepresented_topics = DataProcessor.get_underrepresented_topics(train_df,underrepresented_threshold)

train_df, remaining_topics = DataProcessor.remove_topics_from_dataset(train_df,train_boolean_mask,underrepresented_topics)
validation_df, _ = DataProcessor.remove_topics_from_dataset(validation_df,validation_boolean_mask,underrepresented_topics)

#Get boolean masks of our new dataset
train_boolean_mask = train_df.iloc[:,9:]
validation_boolean_mask = validation_df.iloc[:,9:]

## One Vs Rest Classifiers

In [7]:
#Train and evaluate a Multinonial Naive Bayes classifier (without stemming)
x_train = train_df["Conversation"]
x_test = validation_df["Conversation"]
stop_words = set(stopwords.words('english'))

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for topic in remaining_topics:
    print('... Processing {}'.format(topic))
    # train the model using X_dtm & y
    NB_pipeline.fit(x_train, train_boolean_mask[topic])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(x_test)
    
    print('Test accuracy is {}'.format(accuracy_score(validation_boolean_mask[topic], prediction)))

... Processing Satisfied users
Test accuracy is 0.8403588585244807
... Processing Bugs
Test accuracy is 0.9192504692520599
... Processing Design & UX
Test accuracy is 0.9251932682212961
... Processing Dissatisfied users
Test accuracy is 0.9310024496548214
... Processing Performance
Test accuracy is 0.9515413737155219
... Processing Use cases
Test accuracy is 0.9519613145420418
... Processing Gaming
Test accuracy is 0.9571024082970127
... Processing Feature Requests
Test accuracy is 0.9590812203734929
... Processing Complexity
Test accuracy is 0.9695670155568988
... Processing Security & Accounts
Test accuracy is 0.9734291986129228
... Processing Update
Test accuracy is 0.9729201794292622
... Processing Pricing
Test accuracy is 0.9736837082047529
... Processing Camera & Photos
Test accuracy is 0.9750453345210448
... Processing Video
Test accuracy is 0.9755988928832755
... Processing Customer Support
Test accuracy is 0.9751153246587981
... Processing Notifications & Alerts
Test accuracy 

In [8]:
#Train and evaluate an SV classifier (without stemming)

x_train = train_df["Conversation"]
x_test = validation_df["Conversation"]
stop_words = set(stopwords.words('english'))

SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

for topic in remaining_topics:
    print('... Processing {}'.format(topic))
    # train the model using X_dtm & y
    SVC_pipeline.fit(x_train, train_boolean_mask[topic])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(x_test)
    
    print('Test accuracy is {}'.format(accuracy_score(validation_boolean_mask[topic], prediction)))

... Processing Satisfied users
Test accuracy is 0.8671714440237966
... Processing Bugs
Test accuracy is 0.9654757738682277
... Processing Design & UX
Test accuracy is 0.9715903668119492
... Processing Dissatisfied users
Test accuracy is 0.9556644291031718
... Processing Performance
Test accuracy is 0.9875544809595012
... Processing Use cases
Test accuracy is 0.9615817771132249
... Processing Gaming


KeyboardInterrupt: 

In [ ]:
#Train and evaluate a Logistic Regression classifier (without stemming)

x_train = train_df["Conversation"]
x_test = validation_df["Conversation"]
stop_words = set(stopwords.words('english'))

LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])

for topic in remaining_topics:
    print('... Processing {}'.format(topic))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(x_train, train_boolean_mask[topic])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(validation_boolean_mask[topic], prediction)))